In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# import sequential model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

# Xception
from keras.applications.xception import Xception

# Batch Normalization
from keras.layers import BatchNormalization
import tensorflow as tf

In [ ]:
size = 249
target_size = (size,size,3)

In [ ]:
# ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

train_datagen1 = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   rescale=1./255)

train_gen1 = train_datagen1.flow_from_directory('/content/Dataset/Train',
                                              target_size=(size,size),
                                              batch_size=32,
                                              class_mode='binary')

In [ ]:
test_datagen1 = ImageDataGenerator(rescale=1./255)

test_gen1 = test_datagen1.flow_from_directory('/content/Dataset/Validation',
                                              target_size=(size,size),
                                              batch_size=32,
                                              class_mode='binary')

In [ ]:
train_datagen2 = ImageDataGenerator(rescale=1./255,
                                    rotation_range=30,
                                    shear_range=0.3,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    #width_shift_range=0.4,
                                    #height_shift_range=0.4,
                                   )

train_gen2 =train_datagen2.flow_from_directory('/content/Dataset/Train',
                                            target_size=(size,size),
                                            batch_size=64,
                                            class_mode='binary')

In [ ]:
valid_datagen2 = ImageDataGenerator(rescale=1./255)

valid_gen2 = valid_datagen2.flow_from_directory('/content/Dataset/Validation',
                                                target_size=(size,size),
                                                batch_size=64,
                                                class_mode='binary')

In [ ]:
# mpimg
import mpimg

fig,ax=plt.subplots(ncols=2,nrows=4,figsize=(20,20))
img1 = mpimg.imread('/content/Dataset/Train/Male/000007.jpg')
ax[0][0].imshow(img1)
ax[0][0].set_title("Second Dataset Examples")
img2 = mpimg.imread('/content/Validation/male/063453.jpg.jpg')
ax[0][1].imshow(img2)
ax[0][1].set_title("First Dataset Examples")
img3 =  mpimg.imread('/content/Dataset/Train/Male/000021.jpg')
ax[1][0].imshow(img3)
img4= mpimg.imread('/content/Validation/male/063495.jpg.jpg')
ax[1][1].imshow(img4)
img5 = mpimg.imread('/content/Dataset/Train/Male/000033.jpg')
ax[2][0].imshow(img5)
img6 = mpimg.imread('/content/Validation/male/063517.jpg.jpg')
ax[2][1].imshow(img6)
ax[3][0].imshow(mpimg.imread('/content/Dataset/Train/Female/000005.jpg'))
ax[3][1].imshow(mpimg.imread('/content/Validation/male/063531.jpg.jpg'))
plt.tight_layout()

In [ ]:

#base
model = Sequential()
model.add(Xception(include_top=False, pooling='avg', weights='imagenet', input_shape=target_size))
model.add(Flatten())
model.add(BatchNormalization())
#head
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.layers[0].trainable = False

In [ ]:
model.summary()
model.compile(optimizer= 'Adam', loss = 'binary_crossentropy', metrics= 'accuracy')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
                                             save_best_only = True,
                                             monitor= 'val_accuracy',
                                             mode = 'max')

In [ ]:
epochs = 12
batch_size=256
history1 = model.fit(train_gen1, epochs= epochs, validation_data= test_gen1,
                    steps_per_epoch= len(train_gen1.filenames)//batch_size,
                    validation_steps= len(test_gen1.filenames)//batch_size,
                    callbacks= [checkpoint]
                    )

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'ro', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'ro', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

In [ ]:
test_loss, test_acc = model.evaluate(test_gen1, steps=50)
print('test acc:', test_acc)
print('test_loss:',test_loss)

In [ ]:
test_datagen2 = ImageDataGenerator(rescale=1./255)

test_gen2 = test_datagen2.flow_from_directory('/content/Dataset/Test',
                                              target_size=(size,size),
                                              batch_size=650,
                                              class_mode='binary')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('xception_v2_{epoch:02d}_{val_accuracy:.3f}.h5',
                                             save_best_only = True,
                                             monitor= 'val_accuracy',
                                             mode = 'max')

In [ ]:
epochs = 5 #should train for 10 epochs
batch_size=512
history2 = model.fit(train_gen2, epochs = epochs, validation_data = valid_gen2,
                    steps_per_epoch= len(train_gen2.filenames)//batch_size,
                    validation_steps= len(valid_gen2.filenames)//batch_size,
                    callbacks= [checkpoint]
                    )

In [ ]:
test_loss, test_acc = model.evaluate(test_gen1, steps=50)
print('test acc:', test_acc)
print('test_loss:',test_loss)

In [ ]:
test_loss, test_acc = model.evaluate(test_gen2, steps=100)
print('test acc:', test_acc)
print('test_loss:',test_loss)

In [ ]:
import os
# import img_to_array
# build the model using SVM 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [ ]:
# build the model using SVM
# SVC
from sklearn.svm import SVC

model = SVC(kernel='linear', probability=True)

# fit the model on the training data
len_train = len(train_gen1.filenames)
len_test = len(test_gen1.filenames)

X_train = np.array(train_gen2.filenames).reshape(len_train, -1)
y_train = np.array(train_gen2.classes)

In [ ]:
X_train.shape

In [ ]:
# flatten the images
X_train = X_train/255
X_train[1]

In [ ]:
len_test = len(test_gen1.filenames)
print(len_test)

In [ ]:
X_test = np.array(test_gen1.filenames).reshape(len_test, -1)
y_test = np.array(test_gen1.classes)

In [ ]:
# fitting the data into a SVM model
model.fit(X_train, y_train)

# convert X_test to a numpy array of type float32
X_test = X_test.astype('float32')

